<a href="https://colab.research.google.com/github/vennelaakkenapally/-Transcript-Summarizer/blob/main/Copy_of_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download YouTube Video's Audio

In [ ]:
!pip in[2]
8s
stall pytube


In [ ]:
! pip install pytube -q

In [ ]:
from pytube import YouTube

In [ ]:
#VIDEO_URL = "https://youtu.be/-pHOhJJjJdE?si=Guo19OAHR9-GjYhG"

In [ ]:
VIDEO_URL = 'https://youtu.be/-pHOhJJjJdE?si=Guo19OAHR9-GjYhG'

In [ ]:
yt = YouTube(VIDEO_URL)

Download the audio from the YouTube video

In [ ]:
yt = YouTube(VIDEO_URL)
audio_streams = yt.streams.filter(only_audio=True)

In [ ]:
! ffmpeg -i audio.mp4 -acodec pcm_s16le -ar 16000 audio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Choose a specific audio stream (e.g., first one)

In [ ]:
audio = audio_streams.first()

In [ ]:

# Download the audio file
audio.download(filename='audio.mp4')

'/content/audio.mp4'

English ASR with HuggingSound

In [ ]:
!pip install huggingsound -q

In [ ]:
from huggingsound import SpeechRecognitionModel

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cpu'

In [ ]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)

INFO:huggingsound.speech_recognition.model:Loading model...


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Audio Chunking

In [ ]:
import librosa

In [ ]:
input_file = '/content/audio.wav'

In [ ]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [ ]:
import soundfile as sf

In [ ]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

In [ ]:
i

7

Audio Transcription / ASR / Speech to Text

In [ ]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav')

In [ ]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav',
 '/content/7.wav']

In [ ]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 8/8 [05:44<00:00, 43.05s/it]


In [ ]:
full_transcript = ' '

In [ ]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [ ]:
len(full_transcript)

2213

Text Summarization

In [ ]:
from transformers import pipeline

In [ ]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
summarized_text = summarization(full_transcript)

In [ ]:
summarized_text[0]['summary_text']

' Each communication channel has a capacity which can be defined as the amount of information that can be transmitted over the channel in a given-time unit . The bandwidth of transmission channel is a significant factor but it is not necessarily the only onetheat cost s involves in data transmission due to retendancy with the message itself . Network software arts headers and trailers to any messages for example to specify the destination or to check for errors in the entire messages all of these activities contribute the delay in transmitting data actually .'

In [ ]:
s=len(str(summarized_text))
print(s)

298
